In [ ]:
import chess
from IPython.core.display import HTML, Javascript
with open('style.html', 'r') as file:
     css = file.read()
HTML(css)

In [ ]:
%run Util/00_imports.ipynb
%run Util/01_functions.ipynb

# Spielen gegen die KI

## Vom Nutzer zu tätigende Einstellungen
Zur Erstellung eines individuellen Spielfeldes werden Dictionaries verwendet. Diese verwenden jeweils als `key` die Figur,
die auf dem Spielfeld platziert werden soll. Als `value` nutzen die Einträge jeweils eine Liste, 
die vom Nutzer mit den String-Bezeichnern der Felder gefüllt werden sollen, auf denen die jeweilige Figur steht.
Diese Feldbezeichnungen folgen dem Format, welches in Notebook `01_chess_introduction` erklärt wurde.

> TODO: Mögliche Formatierung der Beschreibung der globalen Variablen
> TODO: Anpassung des globalen Variablen Stils (in gName)

In [ ]:
WHITE_POSITIONS = {chess.KING:['b4'],
                   chess.QUEEN:['a1'],
                   chess.ROOK:[],
                   chess.BISHOP:[],
                   chess.KNIGHT:[],
                   chess.PAWN:['g7']}

BLACK_POSITIONS = {chess.KING:['b1'],
                   chess.QUEEN:[],
                   chess.ROOK:[],
                   chess.BISHOP:[],
                   chess.KNIGHT:[],
                   chess.PAWN:[]}

Weiter muss der Nutzer angeben, welche vorher berechnete Spielsituation er laden möchte.
Hierfür muss der Dateiname in einer globalen Variable (ohne Dateiendung `.chessAI`) angegeben werden.
Bei dieser Variablen handelt es sich um ``FILE``.
Die Dateien werden nach der Berechnung in dem Ordner `S_n_Results` hinterlegt.

In [ ]:
FILE = "s_n_queen_24_05"

## Logik für die Interaktion mit der Spielsituation
Im folgenden Abschnitt sollen einige Hilfsfunktionen erklärt werden, welche für die Interaktion zwischen dem Nutzer über ein Jupyter Notebook und 
der Spielsituation benötigt werden.  

Die Funktion `get_occupied_cells()` übersetzt die zuvor erstellten und vom Nutzer veränderten `Position-Dictionaries` (`WHITE_POSITIONS`, `BLACK_POSITIONS`) in eine Liste, die alle ``values`` der
Dictionaries, also alle besetzten Felder, enthält. Diese Liste wird benötigt, um zu überprüfen, ob die Eingaben des Nutzers korrekte Zellen sind. Sie stellt den Rückgabewert der Funktion dar.

In [ ]:
def get_occupied_cells():
    cells = []
    for values in WHITE_POSITIONS.values():
        for value in values:
            cells.append(value)
    for values in BLACK_POSITIONS.values():
        for value in values:
            cells.append(value)
    return cells

Mit dem Rückgabewert von der vorherigen Funktion übernimmt die Funktion `check_for_correct_cells()` die Überprüfung.
Hierzu wird jeder Wert in den eingegebenen Feldern in den Dictionaries `WHITE_POSITIONS` und `BLACK_POSITIONS` auf Einhaltung
eines regulären Ausdrucks, welcher auf eine Kombination aus Buchstabe und Zahl prüft, überprüft.
Der reguläre Ausdruck `[a-h][1-8]` beschreibt ein Symbol im Bereich a-h (`[a-h]`) sowie ein weiteres Symbol im Bereich 1-8 (`[1-8]`).
Das Ergebnis wird in Form eines booleschen Werts zurückgegeben.

In [ ]:
def check_for_correct_cells():
    cells = get_occupied_cells()
    for cell in cells:
        x = re.search("[a-h][1-8]", cell)
        if x is None:
            print(cell)
            print("Value incorrect!")
            return False
        else:
            pass
    return True

Bevor die Figuren dem Objekt hinzugefügt werden, werden sie in einem Dictionary gesammelt, welches als `piece_map` verwendet wird.

Die Funktion `collect_cells(color, pieces)` erstellt diese `piece_map`.
Diese erhält folgende Parameter:
* ``color``: Ein Boolescher-Wert, der die Farbe der Pieces repräsentiert. Kann auch in Form von `chess.Color` übergeben werden.
* ``pieces``: Ein Dictionary, das als `key` einen `chess.PieceType` nimmt. Der `value` ist eine Liste von Strings, die die besetzten Felder repräsentieren.


Das Dictionary entsteht durch Iteration über `pieces`. Das zurückgegebene Dictionary nutzt als `key` ein `chess.Square`. Der `value` des Dictionaries ist nun ein `chess.Piece`.

In [ ]:
def collect_cells(color, pieces):
    occupied_cells = {}
    for piece_type, values in pieces.items():
            piece = chess.Piece(piece_type, color)
            for value in values:
                square = chess.parse_square(value)
                occupied_cells[square] = piece
            
    return occupied_cells

Die Funktion `create_board()` füllt das Schachbrett letztlich mit den Figuren, die vom Nutzer in den Dictionaries angegeben wurden.
Als Rückgabewert gibt die Funktion das gefüllte Schachbrett als `chess.Board` zurück.

Zunächst werden die vorherigen Funktionen verwendet, um die `piece_map` zu erstellen. Diese 
wird auf das Board-Objekt angewandt und die entstehende Stellung auf Validität überprüft.
Ist das Board nicht valide, wird ein Fehler ausgegeben und ein neues chess.Board wird erstellt.

Ein Problem, welches in Spielsituationen mit dem Turm auftritt, ist die Berechtigung zu einer Rochade.
Damit das Board als valide anerkannt wird, werden die entsprechenden Flags mit der Funktion
`clean_castling_rights` den Regeln entsprechend gesetzt.

> TODO: Wird Beschreibung der Funktion benötigt? @Lukas
> Possible TODO: Anzeigen, weshalb das Board invalid ist.

In [ ]:
def create_board():
    local_board = chess.Board(None)
    occupied_cells = {}
    if check_for_correct_cells():
        occupied_cells |= collect_cells(chess.WHITE, WHITE_POSITIONS)
        occupied_cells |= collect_cells(chess.BLACK, BLACK_POSITIONS)

        local_board.set_piece_map(occupied_cells)
        local_board.castling_rights = local_board.clean_castling_rights()
        
        if not local_board.is_valid():
            display(local_board)
            print("Specified lineup is invalid")
            print("Default board created instead")
            local_board = chess.Board()
    return local_board

## Import der Daten

Für die Bestimmung der Züge der KI werden die $S_n$ Mengen verwendet, die im Notebook ``02_calculation.ipynb`` berechnet werden.
Eine Erklärung worum es sich hierbei handelt, findet sich in diesem Notebook.
Die Ergebnisse der Berechnung werden mittels `pickle` serialisiert und in einer ZIP-Datei komprimiert abgespeichert. Zur Verwendung wird das Archiv entpackt und 
die Liste deserialisiert. 
Weitere Informationen zum Inhalt der Datei befinden sich ebenfalls im Notebook ``calculation.ipynb``.   

Die Funktion `load_data(filename)` erhält den Dateinamen (``filename``) als Parameter und gibt die Endspieldaten zurück.
Die Rückgabe besteht aus einer Liste von verwendeten Figuren `piece_list` und einer Liste von Mengen `s_n_sequence_integers`, welche die Situationen in einem n als Integer darstellen. Diese Mengen enthalten jedoch nur die einzigartigen (ungespiegelten) Spielsituationen.

> TODO: Fehlt nicht eine Beschreibung von n?

In [ ]:
def load_data(filename):
    s_n_sequence_integers = []
    with ZipFile("S_n_Results/" + filename + ".chessAI") as zipped:
        with zipped.open(filename + ".pickle") as calculation:
            tmp = pickle.loads(calculation.read())
            piece_list = tmp[0]
            for item in tmp[1:]:
                s_n_sequence_integers.append(item)
    return piece_list, s_n_sequence_integers

Für die Erstellung der gesamten $S_n$ Mengen werden Funktionen zum Spiegeln der vorhandenen Spielsituationen verwendet. Diese können aus dem Notebook `12_mirroring.ipynb` entnommen werden.

In [ ]:
# In Future (after merge) use following line:
# %run 12_mirroring.ipynb

In [ ]:
SWAPS = {
        "vertical" : {x:x^56 for x in range(64)},
        "horizontal" : {x:x^7 for x in range(64)},
        "rotate_right" : {x:(((x >> 3) | (x << 3)) & 63) ^ 56 for x in range(64)},
        "rotate_180" : {x : x ^ 63 for x in range(64)},
        "rotate_left" : {x : (((x >> 3) | (x << 3)) & 63) ^ 7 for x in range(64)},
        "diagonal" : {x : ((x >> 3) | (x << 3)) & 63 for x in range(64)},
        "anti_diagonal" : {x : (((x >> 3) | (x << 3)) & 63) ^ 63 for x in range(64)}
    }

In [ ]:
# Mirroring mit 7 Bit Darstellung
def mirror_board(board_int, mirror : dict):
    # Save turn
    result = board_int & 1
    # Get count of pieces saved in int
    n = len(PIECE_LIST)
    # Remove turn
    board_int = board_int >> 1

    for i in range(n):
        result |= mirror[board_int & 127] << 7 * i + 1
        board_int = board_int >> 7
    return result

In [ ]:
def mirror_all_directions(board_int):
    result = set()
    for name, swap in SWAPS.items():
        result.add(mirror_board(board_int, swap))
    return result

Die Erstellung der vollständigen $S_n$ Mengen geschieht durch die Funktion `gen_all_integers()`.
Diese nimmt die bereits gespeicherten Mengen in ``S_N_INTEGERS`` und iteriert über diese.
Für jedes Brett der Menge werden die 7 Spiegelungen generiert.
Diese werden in einer Liste von Mengen, `result`, gespeichert.
Die Indizes der Mengen werden dabei nicht verändert. Die Liste `result` stellt den Rückgabewert der Funktion dar.

In [ ]:
def gen_all_integers(s_n_integers):
    result = []
    for sequence in s_n_integers:
        int_set = set()
        for int_board in sequence:
            int_set.add(int_board)
            int_set |= mirror_all_directions(int_board)
        result.append(int_set)
    return result

## Den besten Halbzug für die KI ermitteln
Um mit den berechneten Endspieldaten gegen einen Spieler zu gewinnen, muss jeder Halbzug, den Weiß macht, optimal sein.
Ein passender Halbzug für die KI besteht darin die Situation von $S_n$ in einen Zustand zu überführen, in dem sie sich in $S_{n-1}$ befindet.

Die Funktion `find_next_move(curr_board, s_index, s_n_sequence)` bestimmt für ein übergebenes Board-Objekt einen solchen Spielzug.
Aus Effizienzgründen wird zusätzlich zum gefundenen Spielzug (`move`) der neue Wert für $n$ zurückgegeben.

Für diese Berechnung benötigt ``find_next_move`` nachkommende Argumente:
* ``curr_board``: Das Board, für welches der nächste Spielzug berechnet werden soll.
* ``s_index``: Das $n$ eines $S_n$, in welchem sich ``curr_board`` befindet. Wird aus Effizienzgründen übergeben.
* ``s_n_sequence``: Die Liste mit allen $S_n$.

Kann kein Spielzug gefunden werden, gibt die Funktion den Wert -1 zurück.

Die für diesen Ablauf benötigte Funktion `find_situation_in_sequence` wird im Notebook `functions.ipynb` definiert und erklärt.

> TODO: Referenz richtig setzen
> TODO: Sind die Übergabeparameter nicht witzlos, da wir die Funktion immer nur mit den globalen Variablen aufrufen? @Lukas xD

In [ ]:
def find_next_move(curr_board, s_index, s_n_sequence):
    s_index_new = s_index - 1
    for move in curr_board.legal_moves:
        curr_board.push(move)
        curr_integer = to_integer(curr_board, PIECE_LIST)
        _tmp = find_situation_in_sequence(curr_integer, [s_n_sequence[s_index_new]])
        if _tmp != -1:
            curr_board.pop()
            return s_index_new, move
        curr_board.pop()

    return -1, None

## Globale Variablen für die Anzeige

Nachdem alle Funktionen für die Bestimmung eines Halbzugs definiert worden sind, gilt es die bereitgestellte UI der ``python-chess`` Bibliothek zu erweitern.
In Form von globalen Variablen werden UI-Elemente definiert, die für die Eingabe eines neuen Halbzugs benötigt werden:

- `INPUT_FIELD`: Ein Eingabefeld, in dem der nächste Halbzug von Schwarz eingetragen werden soll.
- `EXECUTE_BUTTON`: Ein Button, der nach der Eingabe des Halbzugs diesen auch auf dem Schachbrett ausführt.

In [ ]:
INPUT_FIELD = widgets.Text()

EXECUTE_BUTTON = widgets.Button(
    description='Execute Move',
    disabled=False,
    button_style='',
    tooltip='Executes the move entered in the input field.',
    icon='check'
)

## Globale Variablen für den aktuellen Spielzustand
Damit der aktuelle Stand des Spiels auch ohne eine lineare Kette von Funktionsaufrufen verfügbar ist,
werden diese Informationen in globalen Variablen gespeichert.
Bei den Variablen handelt es sich um:
* `PIECE_LIST`: Liste der verwendeten Figuren. Wird für die Integer Konversion verwendet.
* `S_N_INTEGERS`: Die berechnete Endspieltabelle.
* `S_INDEX`: $n$, in welchem $S_n$ sich die aktuelle Stellung befindet.

> TODO: Wirklich eine EndspielTABELLE ?

In [ ]:
PIECE_LIST, S_N_INTEGERS = load_data(FILE)
S_N_INTEGERS = gen_all_integers(S_N_INTEGERS)
S_INDEX = 0

## Hilfsfunktionen

Innerhalb der python-chess library werden die Farben des Schachbretts `chess.WHITE` und `chess.BLACK` als boolesche Variablen definiert.
Deswegen wurde zur späteren Dokumentation eine Funktion geschrieben,
die die Farbe, die gerade am Spielzug ist, als String zurückgibt.

In [ ]:
def get_color(turn):
    if turn:
        return "White"
    else:
        return "Black"

Zum Definieren der globalen Variablen `FILENAME` wird die Funktion `update_filename` definiert.

> TODO: Wird FILENAME in diesem Kontext wirklich global benötigt?
> Antwort: Wenn wir FILENAME nicht global definieren müssen wir halt wieder überall filename übergeben...

In [ ]:
def update_filename(new_value):
    global FILENAME
    FILENAME = new_value

Weiter werden für den Spielbeginn (`reset_board()`) und auch für ausgeführte Spielzüge (`reset_input_field()`) Funktionen geschrieben,
die die UI Elemente auf ihren Standardwert zurücksetzen.

`reset_input_field()` setzt hierbei nur den Value des `INPUT_FIELD` auf einen leeren String.

In [ ]:
def reset_input_field():
    global INPUT_FIELD
    INPUT_FIELD.value = ''

`reset_board()` setzt das `BOARD` Objekt auf den Ausgangszustand zurück und passt dementsprechend die globalen Variablen an.

In [ ]:
def reset_board():
    global BOARD
    global S_INDEX
    BOARD = create_board()
    integer = to_integer(BOARD, PIECE_LIST)
    S_INDEX = find_situation_in_sequence(integer, S_N_INTEGERS)

Zum Durchführen der Spielzüge im Board-Objekt wird die Hilfsfunktion `execute_move(move)` verwendet.
Die Funktion erhält als Parameter:
 * `move`: Ein `chess.Move`, der ausgeführt werden soll.

Zusätzlich wird in der Datei, die in unter dem Namen `FILENAME` zu finden ist, für den Halbzug ein Eintrag hinterlegt.

In [ ]:
def execute_move(move):
    global BOARD
    global S_INDEX
    global S_N_INTEGERS
    turn = BOARD.turn
    move_file = open("Played_Games/" + FILENAME, "a")
    if turn:
        move_file.write(str(BOARD.fullmove_number) + ". " + move.uci() + " ")
    else:
        move_file.write(move.uci() + "\n")
    move_file.close()
    BOARD.push(move)
    integer = to_integer(BOARD, PIECE_LIST)
    S_INDEX = find_situation_in_sequence(integer, S_N_INTEGERS)

Gleichermaßen wurde eine Funktion zum Durchführen der Spielzüge der KI definiert.
Die Funktion `execute_ai_move()` nutzt hierfür sowohl die Funktion `find_next_move(curr_board, s_index, s_n_sequence)` als auch `exeucute_move(move)` verwendet.
Dadurch ist mit der Funktion `execute_ai_move()` die Bestimmung und die Durchführung eines Halbzugs der KI gegeben.

In [ ]:
def execute_ai_move():
    global S_INDEX
    S_INDEX, next_move = find_next_move(BOARD, S_INDEX, S_N_INTEGERS)
    if S_INDEX == -1:
        print("No Move for white found")
    else:
        execute_move(next_move)
        reload_screen()

Da die Anzeige des UI auf Notebook-Zellenausgaben basiert, muss diese auch nach einem neuen Halbzug geleert werden,
sodass die Zelle nicht mit den Elementen überflutet wird. Hierfür aktualisiert die Funktion `reload_screen()` 
die Ausgabe und zeigt erneut das Schachbrett, das Eingabefeld für den nächsten Halbzug und den Knopf zum Ausführen des Halbzugs an.

In [ ]:
def reload_screen():
    clear_output()
    display(BOARD, INPUT_FIELD, EXECUTE_BUTTON)
    display(Javascript("setTimeout(function focus() {document.querySelector('input').focus()}, 100);"))

Die Funktion `show_end_screen()` hingegen zeigt nur das Schachbrett an.

In [ ]:
def show_end_screen():
    clear_output()
    display(BOARD)

Die `game_result()` Funktion gibt den Grund, weshalb das Spiel beendet wurde, als String zurück.
Hierzu wird nur abgefragt, ob die Partie mit einem Schachmatt beendet wurde.
Dies würde bedeuten, dass die Schach-Endspiel-KI gewonnen hat und demnach die Aufgabe der Studienarbeit erfolgreich bearbeitet wurde.
In jeglichem anderen Fall wurde die Aufgabe nicht erfüllt.
Aus diesem Grund müssen andere Spielausgänge nicht überprüft werden.

In [ ]:
def game_result():
    if BOARD.is_checkmate():
        return get_color(BOARD.turn) + " has lost because of Checkmate!"
    else:
        return "Something went wrong!"

Für den `EXECUTE_BUTTON` und das `INPUT_FIELD` wird eine Funktion geschrieben, die für das Ausführen des Halbzugs verantwortlich ist.
Diese wird entweder durch Klicken des Buttons oder Drücken der Enter-Taste aufgerufen. 
Das zuvor beschriebene Verhalten löst die Funktion `execute_entered_move(change)` aus.
Für die Definition dieser Funktion werden zuvor jedoch zwei Hilfsfunktionen definiert.

Die Funktion `evaluate_upcoming_board_situation()` überprüft, ob das `BOARD` Objekt sich in einem beendeten Zustand befindet. Dieser wird mithilfe von `game_result()` ausgewertet.

> TODO: Wenn wir gleichermaßen wie bei den anderen Funktionen argumentieren sollte hier doch ein BOARD.is_checkmate() reichen, oder? Falls wir es so machen möchten is die Funktion game_result() unnötig @Lukas

In [ ]:
# TODO: Besseren Namen ausdenken xD
def evaluate_upcoming_board_situation():
    if BOARD.legal_moves.count() == 0:
        show_end_screen()
        print(game_result())

Die Eingabe, die der Nutzer in Form des `INPUT_FIELD` tätigt, wird mit der Funktion `evaluate_input_field()`  ausgewertet.
Sofern es sich bei der Eingabe um einen Spielzug handelt, wird dieser auf Validität überprüft.
Ein valider Halbzug wird ausgeführt. Anschließend wird der nächste Halbzug der KI durchgeführt.
Andernfalls wird der Nutzer zur erneuten Eingabe aufgefordert.

In [ ]:
def evaluate_input_field():
    black_move = chess.Move.from_uci(INPUT_FIELD.value)
    if black_move in list(BOARD.legal_moves):
        execute_move(black_move)
        reset_input_field()
        execute_ai_move()
        evaluate_upcoming_board_situation()
    else:
        print("Entered an invalid move. Please try again!")
        print(INPUT_FIELD.value)
        time.sleep(2)
        reload_screen()

Die Funktion `execute_entered_move(change)` wird mit der Betätigung des Buttons oder dem Drücken der Enter-Taste aufgerufen. Diese nutzt die Funktionen `evaluate_upcoming_board_situation()` und `evaluate_input_field()` zur Auswertung der Eingabe. Bei einer invaliden Eingabe (kein Spielzug) wird auch hier der Nutzer zur erneuten Eingabe aufgefordert.

In [ ]:
def execute_entered_move(change):
    try:
        if INPUT_FIELD.value != '':
            evaluate_input_field()
        else:
            print("Enter a move!")
            time.sleep(2)
            reload_screen()
    except ValueError:
        print("Entered a wrong move. Please try again!")
        print(INPUT_FIELD.value)
        time.sleep(2)
        reload_screen()

Damit die Funktion beim Klicken des `EXECUTE_BUTTON` oder durch Drücken der Enter-Taste ausgeführt wird, muss diese den `on_click` Events zugewiesen werden.

In [ ]:
EXECUTE_BUTTON.on_click(execute_entered_move)
INPUT_FIELD.on_submit(execute_entered_move)

Für eine Wiederholung einer Partie wurden Funktionen im Notebook `04_play_from_history.ipynb` definiert. Diese nutzen eine angegebene Datei, die sich im Ordner `Played_Games` befindet. Diese Dateien sind wie folgt aufgebaut:
1. Die erste Zeile des Dokuments beinhaltet die FEN der Ausgangssituation
2. Die nachstehenden Zeilen haben folgendes Format:
    * \[Zugnummer\] \[Halbzug weiß\] \[Halbzug schwarz\]
    * Die Züge werden in Form von zwei aufeinander folgenden Feldern beschrieben. Beispiel: `e2e4`

Zur Erstellung dieser Datei wurde die Funktion `create_move_history()` definiert.
Sie fügt außerdem in die erste Zeile die FEN, die die Spielsituation beschreibt, ein.
Die Datei trägt den Namen gespeichert in `FILENAME`.

In [ ]:
def create_move_history():
    update_filename("Move-History_" + str(datetime.today().replace(microsecond=0)).replace(":","_") + ".txt")
    move_file = open("Played_Games/" + FILENAME, "a")
    move_file.write(BOARD.fen() + "\n")
    move_file.close()

Die Funktion `start_game()` bereitet alle Parameter für einen Spielverlauf vor. Dazu gehört:
- Erstellung eines neuen Boards.
- Das Erstellen einer neuen Historie für das neu begonnene Spiel.
- Anzeigen der neuen Spielsituation.

Die Implementierung sieht dabei vor, dass Weiß (also die KI) immer den ersten Halbzug hat.

In [ ]:
def start_game():
    global BOARD
    reset_board()
    create_move_history()
    execute_ai_move()

Mit dem Aufruf der Funktion `start_game` kann nun ein Spiel gegen die KI gestartet werden.

> TODO: Folgenden Fehler abfangen: Wenn man in seiner aktuellen Position einen Turm bspw. auf dem Feld hat, muss man auch darauf achten, dass bei der verwendeten chessAI Datei (und damit auch in der Piece_List) auch die gleichen Figuren verwendet werden.

In [ ]:
start_game()